In [6]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt-get -qq install -y python-libtorrent libtorrent-rasterbar-dev #for utilities used inside
!apt-get -qq install -y zip
!pip install python-libtorrent

In [18]:


import tensorflow as tf
import requests
import libtorrent as lt
from google.colab import drive
import re
from google.colab import files
import time
import os


def getFilename_fromCd(cd):
    """
    Get filename from content-disposition
    """
    if not cd:
        return None
    fname = re.findall('filename=(.+)', cd)
    if len(fname) == 0:
        return None
    return fname[0]

drive.mount('/content/drive')  #mounts your Drive 

start_time = time.time()
# set flagsto  true if you want to download from torrent and/or url (file server)
tor_flag = False
url_flag = True
urlName =""
torName =""
url_link ="http://files.pushshift.io/reddit/comments/RC_2017-01.bz2"
tor_link = "magnet:?xt=urn:btih:F6E6F69DDEA8A224BA5B97721AC3259F7B03EEAE&dn=%5BRepoAnime%5D+Yosuga+no+Sora+-+Ep01-12+%2B+Extras%5BWith+EP08+Fix%5D%5BBD+Uncensored%5D%5B1080p+x264+AAC+Eng+Sub%5D&tr=http%3A%2F%2Fnyaa.tracker.wf%3A7777%2Fannounce&tr=http%3A%2F%2Fanidex.moe%3A6969%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.zer0day.to%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fcoppersurfer.tk%3A6969%2Fannounce"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

if url_flag:
    r = requests.get(url_link, allow_redirects=True)
    filename = getFilename_fromCd(r.headers.get('content-disposition'))
    print("filename:",r)
    open(filename, 'wb').write(r.content)
    # !wget $url_link
    urlName = filename

if tor_flag:
    #torrenting 
    ses= lt.session()   #get session handle
    ses.listen_on(6896,6884)

    tparams = {"save_path":".",
            "storage_mode":lt.storage_mode_t(2),
            "paused":False,
            "auto_managed":True,
            "duplicate_is_error":False} #Torrent physical Parameters


    tor_handle = lt.add_magnet_uri(ses, tor_link, tparams) #initializing torrent with parameters
    ses.start_dht()
    print("Fetching Metadata...")    #torrent activity starts
    while(not tor_handle.has_metadata()):
        time.sleep(1)
    torName  = tor_handle.get_torrent_info()

    print("Starting the Torrent...")
    while(tor_handle.status().state != lt.torrent_status.seeding ):   #download progress 
        s = tor_handle.status()
        tor_status = ['queued','checking','fetching metadata',\
                    'downloading','finished', 'seeding', 'allocating']
        print("Progress:",s.progress*100, \
            "Down_speed:",s.download_rate/1000 ,"UP_Speed", s.upload_rate/1000,"Peers:", s.num_peers,"Status:",tor_status[s.state])
        time.sleep(5)
    #/torrenting
    print("---Torrent Downloaded in  %s seconds ---" % (time.time() - start_time))
    print("Downloading Done beero !")

    print("Loading into your drive")
    pathstr= (torName.files().file_path(0)).split("/")
    torName=pathstr[0]
#!mv $sostr "/content/drive/My Drive/"

Wget is non Ram intensive download, Try this in case of failure with previous method

In [23]:
# wget method works primarily for direct download link without any redicrects .
url_link = "http://files.pushshift.io/reddit/comments/RC_2017-01.bz2"
urlName = os.path.basename(url_link)   # it will not work with dowload that have redirects
!wget "$url_link"


RC_2017-01.bz2


In [11]:
 print("Loading into your drive")
 print(torName)
 print(urlName)
if tor_flag :
    !rsync -ar "$torName" "/content/drive/My Drive/"
if url_flag :
    !rsync -ar "$urlName" "/content/drive/My Drive/"

print("Please check your drive for content !!!")


#!mv $sostr "/content/drive/My Drive/"



Loading into your drive


rsync error: received SIGINT, SIGTERM, or SIGHUP (code 20) at rsync.c(644) [sender=3.1.2]
rsync error: received SIGINT, SIGTERM, or SIGHUP (code 20) at io.c(513) [generator=3.1.2]
Please check your drive for content !!!
